In [1]:
!pip install -U \
    langgraph \
    langchain \
    langchain-groq \
    langchain-community \
    duckduckgo-search \
    ddgs \
    requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langgraph
    Found exis

In [20]:
import os

os.environ["GROQ_API_KEY"] = "gsk_Hok*********************k9olc"
os.environ["OPENWEATHER_API_KEY"] = "9007f4a7ecabee***********eb8c"  # https://home.openweathermap.org/api_keys


In [3]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage


In [4]:
class GraphState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    route: str


In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)


In [6]:
def router_agent(state: GraphState):
    query = state["messages"][-1].content

    prompt = f"""
You are a routing agent.

Classify the user query into exactly ONE category:
- general
- math
- search
- weather

Query: {query}

Reply with only one word.
"""

    route = llm.invoke(prompt).content.strip().lower()
    return {"route": route}


In [7]:
def general_agent(state: GraphState):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}


In [8]:
from langchain_core.tools import tool
import re

@tool
def math_tool(expression: str) -> str:
    """Evaluate a mathematical expression"""
    return str(eval(expression))


def math_agent(state: GraphState):
    query = state["messages"][-1].content

    tokens = re.findall(r"[\d\.]+|[\+\-\*/\(\)]", query)

    if not tokens:
        return {"messages": ["No valid math expression found."]}

    expression = "".join(tokens)

    try:
        result = math_tool.invoke(expression)
    except Exception as e:
        return {"messages": [f"Math error: {str(e)}"]}

    return {"messages": [result]}


In [9]:
!pip install -U ddgs

In [10]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

def search_agent(state: GraphState):
    query = state["messages"][-1].content
    result = search_tool.invoke(query)
    return {"messages": [result]}


In [11]:
import requests
import os

@tool
def weather_tool(city: str) -> str:
    """Get current weather using OpenWeatherMap API"""

    api_key = os.getenv("OPENWEATHER_API_KEY")
    if not api_key:
        return "Weather API key not configured."

    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric"
    }

    response = requests.get(url, params=params, timeout=10)
    data = response.json()

    if response.status_code != 200:
        return f"Weather error: {data.get('message', 'Unknown error')}"

    return (
        f"Weather in {city}:\n"
        f"- Temperature: {data['main']['temp']}°C\n"
        f"- Feels like: {data['main']['feels_like']}°C\n"
        f"- Condition: {data['weather'][0]['description']}\n"
        f"- Humidity: {data['main']['humidity']}%"
    )


In [12]:
import re

def weather_agent(state: GraphState):
    query = state["messages"][-1].content

    match = re.search(r"in\s+([a-zA-Z\s]+)", query.lower())

    if not match:
        return {"messages": ["Please specify a city name."]}

    city = match.group(1).strip().title()
    result = weather_tool.invoke(city)

    return {"messages": [result]}


In [13]:
def route_decision(state: GraphState):
    return state["route"]


In [14]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

graph = StateGraph(GraphState)

graph.add_node("router", router_agent)
graph.add_node("general", general_agent)
graph.add_node("math", math_agent)
graph.add_node("search", search_agent)
graph.add_node("weather", weather_agent)

graph.add_edge(START, "router")

graph.add_conditional_edges(
    "router",
    route_decision,
    {
        "general": "general",
        "math": "math",
        "search": "search",
        "weather": "weather",
    }
)

graph.add_edge("general", END)
graph.add_edge("math", END)
graph.add_edge("search", END)
graph.add_edge("weather", END)

app = graph.compile(checkpointer=memory)


In [15]:
config = {"configurable": {"thread_id": "user-1"}}


In [16]:
app.invoke(
    {"messages": [HumanMessage(content="What is 56 * 23?")]},
    config=config
)["messages"][-1].content


'1288'

In [17]:
app.invoke(
    {"messages": [HumanMessage(content="Who is the CEO of OpenAI?")]},
    config=config
)["messages"][-1].content


'Who is the CEO of OpenAI ? ... the drama, you ’ re probably itching to discover just who Sam Altman is , what he brings to the table, and what ... Murati, who is 35 years old at the time of article creation, will serve as interim CEO at OpenAI , the largest company in the AI industry, but ... Meanwhile, hundreds of OpenAI employees said they would leave for jobs at Microsoft, OpenAI ’s lead investor, unless the board reinstated Altman. Today I got a call inviting me to consider a once-in-a-lifetime opportunity: to become the interim CEO of @ OpenAI , ” Shear wrote on X . OpenAI has one of those – one who also happens to have experience being the CEO of a public company previously, in Sarah Friar .'

In [21]:
app.invoke(
    {"messages": [HumanMessage(content="Today's weather in Bangalore")]},
    config=config
)["messages"][-1].content


'Weather error: Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'

In [22]:
app.invoke(
    {"messages": [HumanMessage(content="My name is Sayandeep")]},
    config=config
)

app.invoke(
    {"messages": [HumanMessage(content="What is my name?")]},
    config=config
)["messages"][-1].content


'Your name is Sayandeep.'